In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
global month, longitudes, latitudes, coords
# longitude <--> 
# latitudes ^|

month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

longitudes = np.arange(0,360,3.75)
latitudes = np.arange(-90,92.5,2.5)

long_mesh, lat_mesh = np.meshgrid(longitudes, latitudes)
coords = np.column_stack((long_mesh.ravel(), lat_mesh.ravel()))

def matgen(file):
    global month 
    var_line = []
    var = []
    weather = {}

    
    with open(file) as f:
        lines = f.readlines()

        for l in lines:
            if "7008" in l:
                pass
            else:
                ts = l.split(' ')
               # ts = [t for t in ts if t != '']
                for item in ts:
                    try:
                        var.append(float(item))
                    except: 
                        break


    monthly_var = []

    local = ()

    for i in range(0, len(var), int(len(var)/len(month))):
        monthly_var.append(var[i : i + int(len(var)/len(month))])

    for i in range(0, 12):
        weather[month[i]] = monthly_var[i]

    weather_var = pd.DataFrame(weather)
    weather_var['Longitude'] = coords[:,0]
    weather_var['Latitude'] = coords[:,1]

    return weather_var  

In [3]:
temps = matgen("weather-data\A2a\HADCM3_A2a_TEMP_2020.dif")
temps

ValueError: range() arg 3 must not be zero

In [10]:
base_map = gpd.read_file('base_map.shp')

# Create a GeoDataFrame from the DataFrame containing the coordinates
gdf = gpd.GeoDataFrame(temps, geometry=gpd.points_from_xy(temps['Longitude'], temps['Latitude']))

map_plot = gpd.read_file(gpd.datasets.get_path('nybb'))

# Create the plot
fig, ax = plt.subplots(figsize=(10, 10))
base_map.plot(ax=ax, color='lightgray')  # Plot the base map
gdf.plot(ax=ax, column='value', cmap='hot', markersize=5)  # Plot the heatmap

# Add labels, title, etc. as desired
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Heatmap Overlay on Base Map')

# Display the plot
plt.show()

DriverError: base_map.shp: No such file or directory